In [68]:
import sys
sys.path.append('../')
import importlib
import models.transunet as vit
import models.decoder_layers as decoder_layers
import models.decoder_layers as encoder_layers
import models.utils as ut
import experiments.config as conf
# import data_processing.dataset_synapse as dp 
import data_processing.data_parser as dp
import data_processing.dataset_synapse as ds
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa
import models.resnet_v2 as res

tfk = tf.keras

In [92]:
input_shape = (1, 2, 2, 4)
x = np.arange(np.prod(input_shape)).reshape(input_shape)
print(x)

[[[[ 0  1  2  3]
   [ 4  5  6  7]]

  [[ 8  9 10 11]
   [12 13 14 15]]]]


In [94]:
y = tf.keras.layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)
print(y)

tf.Tensor(
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 0  0  0  0]]

  [[ 8  9 10 11]
   [12 13 14 15]
   [ 0  0  0  0]]

  [[ 0  0  0  0]
   [ 0  0  0  0]
   [ 0  0  0  0]]]], shape=(1, 3, 3, 4), dtype=int32)


In [82]:
dummy.shape

(1, 224, 224, 3)

In [97]:
importlib.reload(res)
model = res.ResNetV2([3,4,9])
model(dummy)

(1, 55, 55, 256)
(1, 55, 55, 256)
(1, 56, 56, 256)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 28, 28, 512)
(1, 14, 14, 1024)
(1, 14, 14, 1024)


(<tf.Tensor: shape=(1, 14, 14, 1024), dtype=float32, numpy=
 array([[[[1.6419067e+00, 2.8190899e-01, 1.3068233e+00, ...,
           4.8643484e+00, 2.9882786e+00, 2.0744565e+00],
          [0.0000000e+00, 6.2139308e-01, 1.5283296e+00, ...,
           2.6190541e+00, 2.2097852e+00, 0.0000000e+00],
          [6.2280470e-01, 6.7247808e-01, 2.5756317e-01, ...,
           5.1654224e+00, 5.8694065e-01, 3.7248759e+00],
          ...,
          [4.1673059e+00, 3.1737103e+00, 2.4511485e+00, ...,
           4.2539368e+00, 2.2410641e+00, 2.0529838e+00],
          [4.9839492e+00, 2.3349152e+00, 4.1883726e+00, ...,
           1.2408931e+00, 1.4827459e+00, 7.3704934e-01],
          [1.5628650e+00, 1.6329346e+00, 7.0511410e-04, ...,
           7.7005374e-01, 2.7080584e+00, 2.6518226e-02]],
 
         [[4.0344685e-01, 5.6608373e-01, 7.7309084e-01, ...,
           1.6268787e+00, 5.2690897e+00, 6.6452593e-01],
          [3.8695159e+00, 4.6216879e+00, 2.6304522e+00, ...,
           2.4257534e+00, 1.3633397

In [98]:
model.summary()

AttributeError: 'ResNetV2' object has no attribute 'summary'

In [85]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
t.shape

TensorShape([2, 3])

In [10]:
X, y = ds.load_data("../data/train_npz/", 10, output_size=224)

In [8]:
print(X.shape)
print(y.shape)

(10, 384, 384, 3)
(10, 384, 384, 9)


In [2]:
resnet = tfk.applications.ResNet50V2(include_top=False, input_shape=(224,224,3))

In [37]:
w = resnet.get_layer('conv1_conv').weights[0]
m, v = tf.nn.moments(w, axes=[0,1,2], keepdims=True) 
w = (w-m) / tf.sqrt(v+1e-5)
tf.math.is_nan(w)
w.shape


TensorShape([7, 7, 3, 64])

In [60]:
class StdConv2D(tfkl.Conv2D):
    
    def call(self, x):
        w = self.weights[0]
        m, v = tf.nn.moments(w, axes=[0,1,2], keepdims=True) 
        w = (w-m) / tf.sqrt(v+1e-5)
        # self.weights = w 
        return tf.nn.conv2d(x, w, self.strides, self.padding.upper(), "NHWC" if self.data_format == "channels_last" else "NCHW", self.dilation_rate, self.name)

In [67]:
conv = StdConv2D(filters=64, kernel_size=7, strides=2, padding="same")
dummy = np.random.rand(1,224,224,3)
# conv(dummy)

In [3]:
resnet.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [4]:
importlib.reload(conf)
importlib.reload(decoder_layers)
importlib.reload(encoder_layers)
importlib.reload(vit)
importlib.reload(ut)
config = conf.get_r50_b16()
trans = vit.TransUnet(config)
trans.compile()

In [8]:
trans.model.fit(x=X, y=y, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000001D3F0393D90>
Traceback (most recent call last):
  File "D:\Logiciel\anaconda\envs\tf-keras-gpu\lib\weakref.py", line 357, in remove
    self = selfref()
KeyboardInterrupt


In [11]:
importlib.reload(conf)
importlib.reload(decoder_layers)
importlib.reload(encoder_layers)
importlib.reload(vit)
importlib.reload(ut)
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()